In [1]:
import numpy as np
import word_embedding as we

In [2]:
import nltk

# Function to check if the dataset is already downloaded
def download_nltk_data_if_needed(dataset_name):
    try:
        # Check if the dataset is already available locally
        nltk.data.find(f'corpora/{dataset_name}')
    except LookupError:
        # If not found, download the dataset
        nltk.download(dataset_name)

# Replace 'treebank' with any other dataset you want to check
download_nltk_data_if_needed('treebank')

In [3]:
tagged_corpus = nltk.corpus.treebank.tagged_sents()
print(tagged_corpus[0])

[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]


In [ ]:
window_sizes = [2, 3, 4, 5]  # Experiment with different window sizes
words_to_check = ['bank', 'life', 'deal']  # Words to check nearest neighbors for

In [4]:


for window_size in window_sizes:
    print(f"\n--- Training with window size: {window_size} ---")
    
    # Train using word2vec_pos with the current window size
    co_occurrence_matrix, vocab_index = we.word2vec_pos(tagged_corpus, window_size=window_size)
    
    # Evaluate by checking the nearest neighbors for selected words
    for word in words_to_check:
        neighbors = we.find_nearest_neighbors(word, co_occurrence_matrix, vocab_index, top_n=5)
        print(f"Nearest neighbors for '{word}' with window size {window_size}: {neighbors}")



--- Training with window size: 2 ---


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U4'), dtype('<U32')) -> None

In [5]:
for window_size in window_sizes:
    print(f"\n--- Training with window size: {window_size} ---")
    
    co_occurrence_matrix, vocab_index = we.word2vec_pos(tagged_corpus, window_size=window_size)
    
    # Apply PPMI transformation
    ppmi_matrix = we.compute_ppmi(co_occurrence_matrix)

    for word in words_to_check:
        word_with_pos = f"{word.lower()}_NN"  # Example POS tag, adjust as necessary
        if word_with_pos in vocab_index:
            word_idx = vocab_index[word_with_pos]
            word_vector = ppmi_matrix[word_idx, :]
            neighbors = we.find_nearest_neighbors(word_vector, ppmi_matrix, vocab_index, top_n=5)
            print(f"Nearest neighbors for '{word}' with window size {window_size}: {neighbors}")
        else:
            print(f"Word '{word}' not found in the vocabulary.")




--- Training with window size: 2 ---


d:\Github_Repos\DSA4212-word-embeddings\word_embedding.py:69: RuntimeWarning: invalid value encountered in divide
  ppmi_matrix = np.maximum(np.log2(joint_prob_matrix / (word_prob[:, None] * context_prob[None, :])), 0)
d:\Github_Repos\DSA4212-word-embeddings\word_embedding.py:69: RuntimeWarning: divide by zero encountered in log2
  ppmi_matrix = np.maximum(np.log2(joint_prob_matrix / (word_prob[:, None] * context_prob[None, :])), 0)


Nearest neighbors for 'bank' with window size 2: [('bank_NN', 0.9999999999999999), ('guys_NNS', 0.20076073137124925), ('hub_NN', 0.1799275534810626), ('rap_NN', 0.16374325243244675), ('banking_NN', 0.15719172981916166)]
Nearest neighbors for 'life' with window size 2: [('life_NN', 1.0), ('wine-making_NN', 0.24138430367839261), ('batting_NN', 0.20128615359452723), ('chaotic_JJ', 0.17092618914232138), ('133.8_CD', 0.16481087836525135)]
Nearest neighbors for 'deal' with window size 2: [('deal_NN', 1.0), ('guided_VBD', 0.23631290993596277), ('reds_NNS', 0.21924150256991745), ('male-only_JJ', 0.17611700919135898), ('loading_VBG', 0.17455273194495294)]

--- Training with window size: 3 ---
Nearest neighbors for 'bank' with window size 3: [('bank_NN', 1.0), ('nasdaq_NNP', 0.14399113234415958), ('leading_JJ', 0.13282857642794185), ('soviet_JJ', 0.12989160450485296), ('banking_NN', 0.12808070676783648)]
Nearest neighbors for 'life' with window size 3: [('life_NN', 1.0), ('inadequacy_NN', 0.2116